# Description: 

Description: In this notebook, we read all the information available from different datasets and we preprocess this information in order to create the categories that will be feed to the model. 

## Install packages necessary to make connection with azure data lake

#import libraries to create connection
from azure.datalake.store import core, lib, multithread
from azure.common.credentials import ServicePrincipalCredentials

## Get authentification for the connection

from azure.common.credentials import ServicePrincipalCredentials
token1 = lib.auth()

# Import libraries and define paths

In [1]:
import pandas as pd
import datetime as dt
import numpy

year = '2015'

#define paths
path_local = '/Users/mendes/Box Sync/ABI-Project/Project'
path_inbound = path_local+f'/NAZ/People/TurnoverModel/Data/Inbound/{year}/'
path_working = path_local+f'/NAZ/People/TurnoverModel/Data/Working/{year}/'

#define paths to read the inputs and save outputs
path_head = path_inbound+'02 SharpOps/head/'
path_movements = path_inbound+'02 SharpOps/movements/'
path_sharps = path_inbound+'02 SharpOps/'
path_navigate = path_inbound+'04 Navigate/'
path_bonus = path_inbound+'01 Bonus and Salary/'

path_to_save = '/Users/mendes/tech_interview/ABI-tech/Input/SharpOps/'

# Define functions

In [3]:
def adlsopen(x):
    """
    A decorator function to open a file using the azure services in datalake.
    
    Args:
        x: filepath to open
    Returns:
        x: the filepath in the datalake
    """
    return x

def save_csv_adls(df,path,adls=False):
    """
    A decorator function to save a pandas dataframe using the azure services in datalake.
    
    Args:
        df (Pandas DataFrame): dataframe to save
        path (String): string with the filepath to save the file
    """
    df.to_csv(path,index=False)

adls=False

def convert_id(col):
    """
    Convert ids with mismatches or missing initial numbers from headcounts
    
    Args:
        col (Pandas Series): pandas series containing the ids
    Returns:
        l (list): list with the fixed ids
    """
    l=[]
    for k in col:
        k=str(k)
        if len(k)==4: k='1000'+k
        elif len(k)==5: k='100'+k
        elif len(k)==6: k='10'+k
        elif len(k)==7: k='1'+k
        l.append(k)
    return l

def headcount_procedure(hdf,idx_left=None,fixYear=True):
    """
    Convert the raw dataset for headcount into a processed dataset to be inserted in the model
    
    Args:
        hdf (Pandas DataFrame): raw headcount dataset
        idx_left (Pandas Series) = indexes for people who left the company
    Returns:
        hdf (Pandas DataFrame): pandas dataframe with headcount dataset processed
    """
    #remove columns that don't have important information or are already represented by others
    cols_remove = ['Original Hire Date', 'Rehire Date','New Hire', 'Personnel number', 'Full Name',
               'Position','CostCenter Description','ABInbev Entity1',
               'ABInbev Entity3', 'ABInbev Entity4','Company Code','Personnel area',
               'Personnel subarea','Employee subgroup','Short Text of Organizational Unit','Gender', 'Ethnicity',
               'Personnel Area Text','Functional Area','Manager Position','Cost Center']
    hdf=hdf.drop(cols_remove,axis=1)

    #remove special characters and transform all to lower case
    hdf.columns = replace_special(list(hdf.columns))

    #remove duplicated rows from the dataframe
    hdf=hdf[~hdf.duplicated('global id')]

    #calculate age fom date of birth
    hdf['age'] = get_age_from_birth(hdf['date of birth'],fixYear=fixYear)
    hdf = hdf.drop(['date of birth'],axis=1)

    #transform age information into percentiles
    hdf['age compared'] = get_stds(hdf['age']) 

    #transform the age information into buckets with 5 years
    hdf['age buckets'] = pd.cut(hdf['age'],bins=range(0,100,5),include_lowest=True).astype(str)
    hdf=hdf.drop(['age'],axis=1)

    #calculate the service year anniversaries based on the service year metric
    for num in [1,2,3,5,10,20]:
        hdf[str(num) + ' - anniversary'] = (hdf['service years']>=num).astype(float)

    #transform service years information into percentiles
    hdf['service years compared'] = get_stds(hdf['service years']) 

    #transform the service years information into buckets with 5 years
    hdf['service years buckets'] = pd.cut(hdf['service years'],bins=range(0,100,5),include_lowest=True).astype(str)

    #convert the global id for the manager to the defined standard 
    hdf['personnel number manager'] = convert_id(hdf['personnel number manager'])

    #add information about pay scale and service years for the manager

    ##make a copy of the original dataframe
    aux=hdf.copy(deep=True)
    l=[]

    ##get a list of all the possible global ids
    gid = list(aux['global id'].astype(str))

    ##loop through all the ids contained in the personal manager column
    for k in aux['personnel number manager'].values:
        try:
            ###check if the global id for the manager is in the general list of global ids and return the index
            idx = gid.index(str(k))
            a=idx
        except Exception as e:
            ###if the manager global id is not present, just return None
            a=None
        l.append(a)

    ##create a new series that contain the indexes for the manager global ids that are presented in the original one
    l=pd.Series(l)

# 	##slice the dataframe to retrive the important information
# 	aux=aux.iloc[l,['service years','pay scale group']]
    aux=aux.reindex(index=l)[['service years','pay scale group']]

    ##identify which columns belong to the managers
    aux.columns = ['manager - '+k for k in aux.columns]

    ##add the original employee global id information in order to perform the join operation
    aux['global id']=hdf['global id'].values

    ##add the manager calculated information to the main dataframe
    hdf=hdf.join(aux.set_index('global id'),on='global id')

    #calculate the gap difference between employee pay scale group and manager pay scale group

    ##slice the dataframe to retrieve the information about pay scale group
    aux = hdf[['pay scale group','manager - pay scale group']].copy(deep=True)

    ##iterate over each one of the columns in the sliced dataframe
    for col in aux.columns:
        ###preprocess the information in each element of the pay scale group
        aux[col+' numerical'] = [k.strip().lower().replace(' ','-').replace('--','-').replace('us-','') if k is not numpy.nan else k for k in aux[col]]

        ###convert each value in pay scale group to a numerical band
        aux[col+' numerical'] = [band_dict[k] if k is not numpy.nan else k for k in aux[col+' numerical']]

    ##calculate the gap in band using the information calculated from pay scale group
    aux['delta band with manager'] = aux['pay scale group numerical']  - aux['manager - pay scale group numerical']

    l=[]
    ##convert the delta to defined buckets in order to categorize the information
    for val in aux['delta band with manager']:
        if val<-4:
            a='bigger than -4'
        elif val<-2:
            a='between -4 and -2'
        elif val<0:
            a='between -2 and 0'
        elif val<2:
            a='between 0 and 2'
        elif val<=4:
            a='between 2 and 4'
        elif val>4:
            a='bigger than 4'
        l.append(a)

    ##create a new column with the information about gap converted in ordinal categories
    aux['delta band with manager categorized'] = l

    ##copy the calculated information into the original dataframe
    hdf['delta band with manager'] = aux['delta band with manager'].values

    #add the information about the label
    if idx_left:
        ##initiate the label column
        hdf['label'] = 0

        ##update the labels for the people who were not in the company
        hdf = hdf.set_index('global id')

        ##assign 1 for people who left the company and reset index
        hdf.reindex(idx_left)[['label']] = 1
        hdf = hdf.reset_index()

    hdf['global id'] = hdf['global id'].astype(float).astype(int).astype(str)
    return hdf

def replace_special(element,enconding=None,verbose=False):
    """
    Fix problems realted to encoding caused by characters with incorrect encoding type.
    
    Args:
        element (unicode, list or Pandas Dataframe'): The element to be fixed. 
        It can be a single unicode element or 
        an entire pandas DataFrame where each element will be fixed
    Returns:
        element (unicode, list or Pandas Dataframe'): The fixed element
    """

    def replace_str(text):
        if pd.isnull(text):
            return None
        #if '?' in text: text = text.replace('?',' ')
        try:
            str(text)
            text.encode('utf-8')
            text = text.lower()
        except:
            try:
                text=str(text)
            except:
                if verbose:
                    print(text)
                return "codec error"
            text = text.lower()

            if '\xcc\xe4' in text: text = text.replace('\xcc\xe4','e')
            if '\x87\xc6' in text: text = text.replace('\x87\xc6','ca')
            if '\x8d\x86' in text: text = text.replace('\x8d\x86','ca')
            if '\x8d\x8b' in text: text = text.replace('\x8d\x8b','ca')
            if '\x8d\x9b' in text: text = text.replace('\x8d\x8b','co')
            if '\xe7\xe3' in text: text = text.replace('\xe7\xe3','ca')
            if '\x87\xc6' in text: text = text.replace('\x87\xc6','ca')
            if '\x87\xe4' in text: text = text.replace('\x87\xe4','co')

            if 'ri\x88' in text: text = text.replace('ri\x88','rie')
            if '\xb5re' in text: text = text.replace('\xb5re','are')
            if '\xa3de' in text: text = text.replace('\xa3de','ude')
            if '\xa1st' in text: text = text.replace('\xa1st','ist')
            if '\xa1di' in text: text = text.replace('\xa1di','idi')
            if '\x88nc' in text: text = text.replace('\x88nc','enc')
            if '\xa2lo' in text: text = text.replace('\xa2lo','olo')
            if '\xa3bl' in text: text = text.replace('\xa3bl','ubl')
            if '\xa1si' in text: text = text.replace('\xa1si','isi')
            if '\xa1mi' in text: text = text.replace('\xa1mi','imi')
            if '\xa1de' in text: text = text.replace('\xa1de','ide')
            if '\xc4a' in text: text = text.replace('\xc4a','a')


            if enconding!=None:
                if '\xa1' in text: text = text.replace('\xa1','i')
                if '\xa2' in text: text = text.replace('\xa2','o')


            if '\xb1' in text: text = text.replace('\xb1','n')
            if '\xf0' in text: text = text.replace('\xf0','-')
            if '\x98' in text: text = text.replace('\x98','-')
            if '\xff' in text: text = text.replace('\xff','-')
            if '\xa6' in text: text = text.replace('\xa6','-')
            if '\xbd' in text: text = text.replace('\xbd','-')
            if '\x8c' in text: text = text.replace('\x8c','i')
            if '\xc3' in text: text = text.replace('\xc3','')
            if '\x8d\x8bo' in text: text = text.replace('\x8d\x8bo','cao')
            if '\xca' in text: text = text.replace('\xca','e')
            if '\x90' in text: text = text.replace('\x90','e')
            if '\x92' in text: text = text.replace('\x92','i')
            if '\x97' in text: text = text.replace('\x97','o')
            if '\x87' in text: text = text.replace('\x87','a')
            if '\xee' in text: text = text.replace('\xee','o')
            if '\xd4' in text: text = text.replace('\xd4','o')

            if '\xec' in text: text = text.replace('\xec','u')
            if '\x86' in text: text = text.replace('\x86','a')
            if '\x88' in text: text = text.replace('\x88','a')
            if '\x84' in text: text = text.replace('\x84','o')
            if '\xed' in text: text = text.replace('\xed','i')
            if '\xf3' in text: text = text.replace('\xf3','o')
            if '\xe1' in text: text = text.replace('\xe1','a')
            if '\xf5' in text: text = text.replace('\xf5','o')
            if '\xe9' in text: text = text.replace('\xe9','e')
            if '\xea' in text: text = text.replace('\xea','e')
            if '\xf6' in text: text = text.replace('\xf6','e')
            if '\xe4' in text: text = text.replace('\xe4','e')
            if '\xe8' in text: text = text.replace('\xe8','e')

            if '\xa0' in text: text = text.replace('\xa0','a')
            if '\xa1' in text: text = text.replace('\xa1','a')
            if '\xc1' in text: text = text.replace('\xc1','a')
            if '\xa2' in text: text = text.replace('\xa2','a')
            if '\xa3' in text: text = text.replace('\xa3','a')
            if '\xa4' in text: text = text.replace('\xa4','a')
            if '\xe3' in text: text = text.replace('\xe3','a')
            if '\xe0' in text: text = text.replace('\xe0','a')

            if '\x82' in text: text = text.replace('\x82','a')
            if '\x83' in text: text = text.replace('\x83','a')
            if '\x81' in text: text = text.replace('\x81','a')
            if '\x80' in text: text = text.replace('\x80','a')

            if '\xa9' in text: text = text.replace('\xa9','e')
            if '\xaa' in text: text = text.replace('\xaa','e')

            if '\x89' in text: text = text.replace('\x89','e')
            if '\x8a' in text: text = text.replace('\x8a','e')
            if '\x8e' in text: text = text.replace('\x8e','e')

            if '\xad' in text: text = text.replace('\xad','i')

            if '\x8d' in text: text = text.replace('\x8d','i')
            if '\xcd' in text: text = text.replace('\xcd','i')

            if '\xb3' in text: text = text.replace('\xb3','o')
            if '\xb4' in text: text = text.replace('\xb4','o')
            if '\xb5' in text: text = text.replace('\xb5','o')
            if '\xd5' in text: text = text.replace('\xd5','o')
            if '\xc5' in text: text = text.replace('\xc5','o')

            if '\x93' in text: text = text.replace('\x93','o')
            if '\x94' in text: text = text.replace('\x94','o')
            if '\x95' in text: text = text.replace('\x95','o')

            if '\xba' in text: text = text.replace('\xba','u')
            if '\xda' in text: text = text.replace('\xda','u')
            if '\xbc' in text: text = text.replace('\xbc','u')
            if '\xfc' in text: text = text.replace('\xfc','u')

            if '\x9a' in text: text = text.replace('\x9a','u')
            if '\x9c' in text: text = text.replace('\x9c','u')
            if '\xfa' in text: text = text.replace('\xfa','u')

            if '\xa7' in text: text = text.replace('\xa7','c')
            if '\x87' in text: text = text.replace('\x87','c')
            if '\xe7' in text: text = text.replace('\xe7','c')
            if '\x8d' in text: text = text.replace('\x8d','c')
            if '\xc7' in text: text = text.replace('\xc7','c')

            if '\xf4' in text: text = text.replace('\xf4','o')
            if '\xe2' in text: text = text.replace('\xe2','a')
            if '\xb0' in text: text = text.replace('\xb0','o')
            if '\xb2' in text: text = text.replace('\xb2','q')
            if '\xc9' in text: text = text.replace('\xc9','e')
            if '\xd3' in text: text = text.replace('\xd3','o')
            if '\xeb' in text: text = text.replace('\xeb','e')
            if '\xc8' in text: text = text.replace('\xc8','e')
            if '\xf1' in text: text = text.replace('\xf1','n')
            if '\xc2' in text: text = text.replace('\xc2','a')
            if '\xc0' in text: text = text.replace('\xc0','a')
            if '\xdf' in text: text = text.replace('\xdf','s')
            if '\xae' in text: text = text.replace('\xae','i')
            if '\xb7' in text: text = text.replace('\xb7','-')
            if '\xf9' in text: text = text.replace('\xf9','u')
            if '\xac' in text: text = text.replace('\xac','o')
            if '\xf2' in text: text = text.replace('\xf2','o')
            if '\xd6' in text: text = text.replace('\xd6','-')
            if '\xb9' in text: text = text.replace('\xb9','-')

            if '\xf8' in text: text = text.replace('\xf8','o')
            if '\xce' in text: text = text.replace('\xce','i')
            if '\xdc' in text: text = text.replace('\xdc','u')
            if '\xa8' in text: text = text.replace('\xa8','-')
            if '\x8b' in text: text = text.replace('\x8b','a')
            if '\x99' in text: text = text.replace('\x99','o')
            if '\x9f' in text: text = text.replace('\x9f','u')
            if '\xe5' in text: text = text.replace('\xe5','a')
            if '\x9b' in text: text = text.replace('\x9b','o')
            if '\xe6' in text: text = text.replace('\xe6','e')
            if '\xcc' in text: text = text.replace('\xcc','a')
            if '\x91' in text: text = text.replace('\x91','e')

            if '\xef' in text: text = text.replace('\xef','o')
            if '\x96' in text: text = text.replace('\x96','n')
            if '\xcb' in text: text = text.replace('\xcb','a')
            if '\x8f' in text: text = text.replace('\x8f','e')
            if '\xbb' in text: text = text.replace('\xbb','-')
            if '\x9d' in text: text = text.replace('\x9d','u')
            if '\xd2' in text: text = text.replace('\xd2','-')
            if '\xab' in text: text = text.replace('\xab','-')
            if '\xbf' in text: text = text.replace('\xbf','o')
            if '\xd0' in text: text = text.replace('\xd0','-')
            if '\xdb' in text: text = text.replace('\xdb','S')
            if '\xd1' in text: text = text.replace('\xd1','-')
            if '\xfb' in text: text = text.replace('\xfb','O')
            if '\xc6' in text: text = text.replace('\xc6','a')
            if '\xb6' in text: text = text.replace('\xb6','a')
            if '\x85' in text: text = text.replace('\x85','a')

        try:
            str(text)
        except:
            print(text)
        return str(text)


    def replace_list(l):
        s=[]
        for x in l:
            s.append(replace_str(x))
        return s

    def replace_df(df):
        for column in df.columns:
            df[column] = replace_list(df[column])
        col = df.columns
        col = replace_list(col)
        df.columns = col
        return df

    if type(element) is str: #or type(element) is unicode: 
        element = replace_str(element)
    elif type(element) is list or 'pandas.core.series.Series' in str(type(element)):
        element = replace_list(element)
    elif 'pandas.core.frame.DataFrame' in  str(type(element)):
        element = replace_df(element)
    else: 
        print('Please use string, unicode, list or pandas Dataframe')

    return element

def get_age_from_birth(col,reference_year=None,fixYear=False):
    """
    Get age based on the information about birth date.
    
    Args:
        col (Pandas Series'): pandas series containing the birth dates 
        reference_year (Int): Year to calculate the birth date if not using current
        fix_year (Boolean): Fix if year is not in correct format or raise error instead
    Returns:
        l (list): list with the calculated ages
    """
    
	l=[]
	for year in col:
		if fixYear:
			year = fix_year(year)
		year = dt.datetime.strptime(year,'%m/%d/%Y')
		year = year.year
		if reference_year is None:
			age = dt.datetime.now().year-year
		else:
			age = reference_year-year
		l.append(age)
	return l

def fix_year(string_date):
	try:
		if string_date is not None:
			s=string_date
			idx = find_all_ch('/',s)
			if int(s[idx[-1]+1:])>17:
				y= '19' + s[idx[-1]+1:]
			else:
				y= '20' + s[idx[-1]+1:]
			s=s[:idx[-1]+1] + y
			return s
		return None
	except Exception as e:
		return '01/01/2000'
    
def get_stds(col,label = ['very below','below','mean','above','very above']):
	sm=col.mean()
	st=col.std()
	stds = [sm-2*st,sm-st,sm+st,sm+2*st]
	l=[]
	for v in col:
		if v<stds[0]: l.append(label[0])
		elif v<stds[1]: l.append(label[1])
		elif v<stds[2]: l.append(label[2])
		elif v<stds[3]: l.append(label[3])
		else: l.append(label[4])
	return l

band_dict = {
			 'ceo':0.0, 'ebm':0.0,
			 '0-a':0.0,'0-b':0.5,
			 'i-a':1.0,'i-b':1.5,
			 'ii-a':2.0,'ii-b':2.5,
			 'iii-a':3.0,'iii-b':3.5,
			 'iv-a':4.0,'iv-b':4.5,
			 'v-a':5.0,'v-b':5.5,
			 'vi-a':6.0,'vi-b':6.5,
			 'vii-a':7.0,'vii-b':7.5,
			 'viii-a':8.0,'viii-b':8.5,
			 'ix-a':9.0,'ix-b':9.5,
			 'x-a':10.0,'x-b':10.5,
			 'xi-a':11.0,'xi-b':11.5,
			 
			 'cra-t1':12.0,'cra-t2':12.0,'cra-t3':12.0,
			 'crb-t1':12.0,'crb-t2':12.0,'crb-t3':12.0,'cr-b':12.0,
			 'crc-t1':12.0,'crc-t2':12.0,'crc-t3':12.0,'crc-t4':12.0,
			 'crd-t1':12.0,'crd-t2':12.0,'crd-t3':12.0,'crd-t4':12.0,
			 'cre':12.0,'cre-t1':12.0,'cre-t2':12.0,'cre-t3':12.0,
			 'crf-t1':12.0,'crf-t2':12.0,'crf-t3':12.0,'crf-t4':12.0,
			 'crg-t1':12.0,'crg-t2':12.0,'crg-t3':12.0,
			 'cr-a':12.0, 'cr-c':12.0, 'cr-d':12.0, 'cr-e':12.0, 'cr-f':12.0,
			 'cr-g':12.0, 'crc':12.0, 'crf':12.0, 'crg':12.0,
			 
			 'usd2':12.0,'usd4':12.0,
			 
			 'qct-1':12.0,'qct-2':12.0,'qct-3':12.0,'qct-4':12.0,
			 'qct-c':12.0,'qct-d':12.0,'qct-i':12.0,'qct-ii':12.0,
			 'qgr-ii':12.0,'qgr-iii':12.0,
			 
			 'cont':12.0,
			 'advsr':12.0,
			 
			 'qppt':12.0,
			 'qrep':12.0,
			 'hourly':12.0,
			}

### Get labels

In [6]:
path_working

'/Users/mendes/Box Sync/ABI-Project/Project/NAZ/People/TurnoverModel/Data/Working/2015/'

In [3]:
#read turnover information from 2014
tk14 = pd.read_csv(adlsopen(path_working+'tk14.csv'),dtype='str')
#define the keys for turnover in 2014
tk14 = set(tk14['id'])

#read turnover information from 2015
tk15 = pd.read_csv(adlsopen(path_working+'tk15.csv'),dtype='str')
#define the keys for turnover in 2015
tk15 = set(tk15['id'])

#read turnover information from 2016
tk16 = pd.read_csv(adlsopen(path_working+'tk16.csv'),dtype='str')
#define the keys for turnover in 2016
tk16 = set(tk16['id'])

# Folder SharpOps

### Headcount

This is the general information about the employees. This dataset can also be called the demographic information about the employees. Here we collect information such as name, global id, age, and location of work.

#### load files

In [4]:
#read headcount information for 2014
h14 = pd.read_csv(adlsopen(path_head+'Headcount_2014_Year_End.csv'),low_memory=False,encoding='latin')

#convert the id to the standard format applied to all ids
h14['Global ID'] = convert_id(h14['Global ID'])

#get the set that contains all ids
hk14 = set(h14['Global ID'])

#### Build dataset for headcount 14 with labels

In [5]:
#get the ids for the people who were not in the company in 2014
idx14=list(hk14.intersection(tk15))

#use procedure to process raw dataset
h14 = headcount_procedure(h14,idx14)

#print information about the distribuitions of the labels in the dataset
print(h14['label'].value_counts())
print(h14['label'].value_counts()/len(h14))

#create a version with the simple name columns
h14_raw = h14.copy(deep=True)

#add information about the dataset in the name of the column in order to identify the source
h14.columns = ['head --- '+k if k!= 'global id' and k!='label' else k for k in h14.columns ]

#save the dataset created
save_csv_adls(replace_special(h14.drop(['head --- personnel number manager'],axis=1)),
              path_to_save+'head14.csv',adls)

0    18424
Name: label, dtype: int64
0    1.0
Name: label, dtype: float64
